Part 1

In [5]:
!pip install beautifulsoup4

In [6]:
!pip install lxml

In [7]:
!pip install html5lib

In [8]:
!pip install requests

In [10]:
from bs4 import BeautifulSoup
import requests

In [95]:
import numpy as np

In [82]:
import pandas as pd

column_names = ['Postcode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns = column_names)
#neighborhoods

,Postcode,Borough,Neighborhood


In [89]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
page = BeautifulSoup(source, 'lxml')
toronto_table = page.table.tbody

for distinct in toronto_table.find_all('tr')[1:]:
    
    postcode = distinct.find_all('td')[0].text
    
    try:
        borough = distinct.find_all('td')[1].a.text
        
    except:
        borough = distinct.find_all('td')[1].text
    
    try:
        neighborhood = distinct.find_all('td')[2].a.text
    except:
        neighborhood = borough
    
    neighborhoods = neighborhoods.append({'Postcode': postcode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood}, ignore_index = True)
    
#neighborhoods.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [105]:
toronto_data = pd.DataFrame(neighborhoods)

In [106]:
toronto_data.drop_duplicates(['Borough','Neighborhood'], inplace = True)

In [107]:
toronto_data.replace('Not assigned', np.nan, inplace = True)
toronto_data.dropna(inplace = True)

In [134]:
toronto_data = toronto_data.groupby(['Postcode','Borough']).aggregate(lambda x: ','.join(x)).reset_index()
#toronto_data.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Scarborough,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1J,Scarborough,Scarborough Village
5,M1K,Scarborough,"Ionview,Kennedy Park"
6,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
7,M1M,Scarborough,"Cliffcrest,Cliffside"
8,M1N,Scarborough,Birch Cliff
9,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He..."


In [132]:
toronto_data.shape

(84, 3)

Part 2

In [136]:
!pip install geocoder

In [137]:
import geocoder

In [143]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
#coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [144]:
coordinates.shape

(103, 3)

In [377]:
toronto_geo_data = pd.merge(toronto_data,coordinates,left_on='Postcode',right_on='Postal Code', how = 'inner', left_index = True)
toronto_geo_data.drop(labels = ['Postal Code'], axis = 1, inplace = True)
#toronto_geo_data = toronto_geo_data[toronto_geo_data['Borough'] == 'West Toronto'].reset_index(drop=True)
toronto_geo_data.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Scarborough,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside",43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff,43.692657,-79.264848
10,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He...",43.757410,-79.273304


Part 3

In [378]:
!pip install geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

In [379]:
from sklearn.cluster import KMeans

In [380]:
!pip install folium
import folium

In [381]:
import json
from pandas.io.json import json_normalize

In [382]:
toronto_geo_data['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [383]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.653963, -79.387207.


In [384]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_geo_data['Latitude'], toronto_geo_data['Longitude'], toronto_geo_data['Borough'], toronto_geo_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [385]:
CLIENT_ID = 'TXGR3LVT4U2Z4UE1DAZB1U1HJCPRC4RMSFBA2UBWYS4DTBBJ' # your Foursquare ID
CLIENT_SECRET = 'WYM5CP430JQ0KVKKAN2EI301OSOSSOA4INGINAFTBVTWSRKO' # your Foursquare Secret
VERSION = '201905021' # Foursquare API version

In [386]:
toronto_geo_data.loc[0,'Neighborhood']

'Rouge,Malvern'

In [387]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [388]:
def getNearbyVenues( names, latitudes, longitudes, radius=1000, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip( names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            #code,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  #'Postcode'  
                  'Neighborhoods', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [389]:
toronto_venues = getNearbyVenues(
                                   #codes = toronto_geo_data['Postcode'],
                                   names=toronto_geo_data['Neighborhood'],
                                   latitudes=toronto_geo_data['Latitude'],
                                   longitudes=toronto_geo_data['Longitude']
                                  )
toronto_venues

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Scarborough,Morningside,West Hill
Woburn
Scarborough Village
Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside
Birch Cliff
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Tam O'Shanter
Agincourt North,Milliken
Upper Rouge
Hillcrest Village
Henry Farm
Bayview Village
York Mills
Newtonbrook,Willowdale
Willowdale West
Parkwoods
North York
Flemingdon Park
Bathurst Manor,Wilson Heights
Northwood Park,York University
CFB Toronto
Downsview West
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
East Toronto,Riverdale
India Bazaar
Lawrence Park
Moore Park
Deer Park,Rathnelly,South Hill
Rosedale
Cabbagetown
Church and Wellesley
Harbourfront,Regent Park
Downtown Toronto
St. James Town
Berczy Park
Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court
Bedford Park
Central Toronto
Forest Hill North
The 

,Neighborhoods,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge,Malvern",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
2,"Rouge,Malvern",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
3,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
4,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
5,"Rouge,Malvern",43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant
6,"Rouge,Malvern",43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop
7,"Rouge,Malvern",43.806686,-79.194353,Lee Valley,43.803161,-79.199681,Hobby Shop
8,"Rouge,Malvern",43.806686,-79.194353,FASTSIGNS,43.807882,-79.201968,Business Service
9,"Rouge,Malvern",43.806686,-79.194353,Tim Hortons / Esso,43.801660,-79.199133,Coffee Shop


In [390]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = '', prefix_sep = '')
#print(toronto_onehot, toronto_venues['Neighborhood'])
toronto_onehot = pd.concat([toronto_venues['Neighborhoods'],toronto_onehot], axis =1)
#'Neighborhood' in list(toronto_venues['Venue Category'].unique())
toronto_onehot.head()

,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [391]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped.head()

,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.020408,0.0,0.0,0.0,...,0.000000,0.020408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Agincourt North,Milliken",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Bathurst Manor,Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.037037,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [392]:
def return_most_common_venues(row, num_top_venues = 5):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [393]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Supermarket,Bakery,Coffee Shop,Pizza Place,Caribbean Restaurant,Filipino Restaurant,Breakfast Spot,Malay Restaurant
1,"Agincourt North,Milliken",Chinese Restaurant,Pizza Place,Park,Noodle House,Korean Restaurant,Pharmacy,Coffee Shop,Dessert Shop,Caribbean Restaurant,Shop & Service
2,"Alderwood,Long Branch",Discount Store,Pizza Place,Pharmacy,Grocery Store,Garden Center,Intersection,Sandwich Place,Donut Shop,Gas Station,Shopping Mall
3,"Bathurst Manor,Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Bank,Sushi Restaurant,Supermarket,Mediterranean Restaurant,Men's Store,Bridal Shop,Middle Eastern Restaurant,Ski Chalet
4,Bayview Village,Bank,Japanese Restaurant,Skate Park,Café,Chinese Restaurant,Trail,Shopping Mall,Intersection,Grocery Store,Park


In [394]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=50).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 2, 1, 1, 2, 2, 1, 2])

In [395]:
try:
    toronto_merged.drop('Cluster Labels', axis = 1,inplace = True)
    neighborhoods_venues_sorted.drop('Cluster Labels', axis = 1,inplace = True)
except:
    None

In [396]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [397]:
#toronto_merged.insert(5, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted.head()

toronto_merged = toronto_geo_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood').reset_index(drop = True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0).astype(int)

toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Coffee Shop,Sandwich Place,Business Service,Caribbean Restaurant,Fruit & Vegetable Store,Spa,Chinese Restaurant,Paper / Office Supplies Store,Park
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,1,Breakfast Spot,Italian Restaurant,Playground,Park,Burger Joint,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
2,M1E,Scarborough,"Scarborough,Morningside,West Hill",43.763573,-79.188711,1,Pizza Place,Fast Food Restaurant,Coffee Shop,Bank,Food & Drink Shop,Supermarket,Beer Store,Sandwich Place,Discount Store,Greek Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Park,Fast Food Restaurant,Indian Restaurant,Electronics Store,Chinese Restaurant,Zoo,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Fast Food Restaurant,Bowling Alley,Train Station,Pizza Place,Sandwich Place,Coffee Shop,Convenience Store,Restaurant,Japanese Restaurant,Donut Shop
5,M1K,Scarborough,"Ionview,Kennedy Park",43.727929,-79.262029,0,Coffee Shop,Chinese Restaurant,Grocery Store,Discount Store,Fast Food Restaurant,Rental Car Location,Bank,Department Store,Asian Restaurant,Sandwich Place
6,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,1,Intersection,Coffee Shop,Fast Food Restaurant,Bakery,Bus Line,Pizza Place,Park,Bank,Diner,Soccer Field
7,M1M,Scarborough,"Cliffcrest,Cliffside",43.716316,-79.239476,1,Pizza Place,Fast Food Restaurant,Beach,Cajun / Creole Restaurant,Sports Bar,Furniture / Home Store,Burger Joint,Park,Event Space,Dumpling Restaurant
8,M1N,Scarborough,Birch Cliff,43.692657,-79.264848,1,Park,College Stadium,Restaurant,Fast Food Restaurant,Skating Rink,Café,Bank,Discount Store,Diner,Thai Restaurant
9,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He...",43.757410,-79.273304,2,Coffee Shop,Asian Restaurant,Furniture / Home Store,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Automotive Shop,Electronics Store,Burger Joint,Bowling Alley


In [398]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters